# Locality Reduction

In [59]:
import itertools
from sympy import *
import re
from object_einsum import object_einsum

sigmas = symbols('sigma1:5')

In [60]:
def reduce_positive_term_locality(sigmas, ancilla_symbol):
    print("Reducing a term: ", prod(sigmas))
    sigma_np1 = ancilla_symbol
    res = 7 + prod(sigmas[2:]) - 3 * sigmas[0] - 3 * sigmas[1] + 6*sigma_np1 + 2 * prod(sigmas[2:]) * sigma_np1 - sigmas[0] * prod(sigmas[2:]) - sigmas[1] *prod(sigmas[2:]) - 4 * sigmas[0] * sigma_np1 - 4 * sigmas[1] * sigma_np1 + prod(sigmas[0:2])
    return res

def reduce_negative_term_locality(sigmas, ancilla_symbol):
    print("Reducing a term: ", prod(sigmas))
    sigma_np1 = ancilla_symbol
    res = 5 - prod(sigmas[2:]) - sigmas[0] - sigmas[1] + 2 * sigma_np1 - 2 * prod(sigmas[2:]) * sigma_np1 + sigmas[0] * prod(sigmas[2:]) + sigmas[1] * prod(sigmas[2:]) - 4 * sigmas[0] * sigma_np1 - 4 * sigmas[1] * sigma_np1 + 3 * sigmas[0] * sigmas[1]
    return res

def separate_to_coeff_and_symbols(expr):
    syms = []
    coeffs = []
    for elem in Mul.make_args(expr):
       if type(elem) == Symbol:
           syms.append(elem)
       else:
           coeffs.append(elem)
    if coeffs:
       coeff = coeffs[0]
    else:
       coeff = 1
    return coeff, syms


def reduce_one_locality(expr, variable_name, maximum_index, to_locality):
    next_variable_index = maximum_index + 1
    reduced_expr = 0
    current_locality = 0
    for term in Add.make_args(expand(expr)):
        coeff, syms = separate_to_coeff_and_symbols(term)
        current_locality = max(current_locality, len(syms))
        if current_locality <= to_locality:
            print("Skipping reduction on term: ", term)
            reduced_expr += term
            continue
        next_ancilla = Symbol(f'{variable_name}{next_variable_index}')
        if coeff > 0 :
            this_reduced_expr = reduce_positive_term_locality(syms, next_ancilla)
        else:
            coeff *= -1
            this_reduced_expr = reduce_negative_term_locality(syms, next_ancilla)
        reduced_expr += coeff * this_reduced_expr
        print(f"Reduced a term into {this_reduced_expr}. Added index is ", next_variable_index)
        next_variable_index += 1
    next_maximum_index = next_variable_index - 1
    reduced_locality = current_locality - 1
    return reduced_expr, next_maximum_index, reduced_locality


def reduce_locality(expr, variable_name, maximum_index, to_locality=2):
    reduced_expr, next_maximum_index, locality = reduce_one_locality(expr, variable_name, maximum_index, to_locality)
    print(f"Reduced locality to {locality}")
    if locality < to_locality:
        return expr
    elif locality == to_locality:
        return reduced_expr
    else:
        return reduce_locality(reduced_expr, variable_name, next_maximum_index)
expr = prod(symbols("sigma0:4"))
reduced_expr, total_variables_count, locality = reduce_one_locality(expr, variable_name='sigma', maximum_index=3, to_locality=3)
print("Variables count ", total_variables_count, ", Locality ", locality)
reduced_expr

Reducing a term:  sigma0*sigma1*sigma2*sigma3
Reduced a term into sigma0*sigma1 - sigma0*sigma2*sigma3 - 4*sigma0*sigma4 - 3*sigma0 - sigma1*sigma2*sigma3 - 4*sigma1*sigma4 - 3*sigma1 + 2*sigma2*sigma3*sigma4 + sigma2*sigma3 + 6*sigma4 + 7. Added index is  4
Variables count  4 , Locality  3


sigma0*sigma1 - sigma0*sigma2*sigma3 - 4*sigma0*sigma4 - 3*sigma0 - sigma1*sigma2*sigma3 - 4*sigma1*sigma4 - 3*sigma1 + 2*sigma2*sigma3*sigma4 + sigma2*sigma3 + 6*sigma4 + 7

In [61]:
reduce_locality(expr, variable_name='sigma', maximum_index=3, to_locality=2)

Reducing a term:  sigma0*sigma1*sigma2*sigma3
Reduced a term into sigma0*sigma1 - sigma0*sigma2*sigma3 - 4*sigma0*sigma4 - 3*sigma0 - sigma1*sigma2*sigma3 - 4*sigma1*sigma4 - 3*sigma1 + 2*sigma2*sigma3*sigma4 + sigma2*sigma3 + 6*sigma4 + 7. Added index is  4
Reduced locality to 3
Skipping reduction on term:  7
Skipping reduction on term:  -3*sigma0
Skipping reduction on term:  -3*sigma1
Skipping reduction on term:  6*sigma4
Skipping reduction on term:  sigma0*sigma1
Skipping reduction on term:  sigma2*sigma3
Skipping reduction on term:  -4*sigma0*sigma4
Skipping reduction on term:  -4*sigma1*sigma4
Reducing a term:  sigma0*sigma2*sigma3
Reduced a term into 3*sigma0*sigma2 + sigma0*sigma3 - 4*sigma0*sigma5 - sigma0 + sigma2*sigma3 - 4*sigma2*sigma5 - sigma2 - 2*sigma3*sigma5 - sigma3 + 2*sigma5 + 5. Added index is  5
Reducing a term:  sigma1*sigma2*sigma3
Reduced a term into 3*sigma1*sigma2 + sigma1*sigma3 - 4*sigma1*sigma6 - sigma1 + sigma2*sigma3 - 4*sigma2*sigma6 - sigma2 - 2*sigma3*

sigma0*sigma1 + 3*sigma0*sigma2 + sigma0*sigma3 - 4*sigma0*sigma4 - 4*sigma0*sigma5 - 4*sigma0 + 3*sigma1*sigma2 + sigma1*sigma3 - 4*sigma1*sigma4 - 4*sigma1*sigma6 - 4*sigma1 + 5*sigma2*sigma3 - 2*sigma2*sigma4 - 4*sigma2*sigma5 - 4*sigma2*sigma6 - 8*sigma2*sigma7 - 8*sigma2 - 2*sigma3*sigma4 - 2*sigma3*sigma5 - 2*sigma3*sigma6 - 8*sigma3*sigma7 - 8*sigma3 + 4*sigma4*sigma7 + 8*sigma4 + 2*sigma5 + 2*sigma6 + 12*sigma7 + 31

In [62]:
import numpy as np
def construct_hamiltonian_expr(E0, h, J, K, L):
    expr = 0
    num_variables = len(h)
    sigmas = np.array(symbols(f'sigma0:{num_variables}'), dtype=np.object)
    expr += np.dot(h, sigmas)
    expr += np.dot(sigmas, np.dot(J, sigmas))
    expr += object_einsum("ijk,i,j,k", K, sigmas, sigmas, sigmas)
    expr += object_einsum("ijkl,i,j,k,l", L, sigmas, sigmas, sigmas, sigmas)
    expr += E0
    return expr

def read_generalized_ising_hamiltonian(path):
    with open(path, "r") as f:
        f.readline()  # discard first line
        compressed_hamiltonian = [
            tuple(line.strip().split())
            for line in f.readlines()
        ]

    num_sites = len(compressed_hamiltonian[0][0])
    hamiltonian_terms = [np.zeros((num_sites,)*i) for i in range(num_sites+1)]

    for sites, val in compressed_hamiltonian:
        num_zs = 0
        site_nums = []
        for i, x in enumerate(sites):
            if x == 'z':
                site_nums.append(i)
                num_zs += 1

        hamiltonian_terms[num_zs][tuple(site_nums)] = float(val)

    return hamiltonian_terms

In [63]:
E0, h, J, K, L = read_generalized_ising_hamiltonian("./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=0.65.inp")
expr = construct_hamiltonian_expr(E0, h, J, K, L)


In [64]:
reduced_expr = reduce_locality(expr, variable_name='sigma', maximum_index=3, to_locality=2)
reduced_expr

Skipping reduction on term:  0.0377511754763638
Skipping reduction on term:  0.18262644006721*sigma1
Skipping reduction on term:  0.191003522368368*sigma3
Skipping reduction on term:  0.125841368144624*sigma2
Skipping reduction on term:  0.125841368144624*sigma0*sigma2
Skipping reduction on term:  0.00837708230115862*sigma0*sigma3
Skipping reduction on term:  0.17297610387297*sigma1*sigma3
Skipping reduction on term:  -0.27288404604934*sigma2*sigma3
Reducing a term:  sigma0*sigma1*sigma3
Reduced a term into sigma0*sigma1 - sigma0*sigma3 - 4*sigma0*sigma4 - 3*sigma0 - sigma1*sigma3 - 4*sigma1*sigma4 - 3*sigma1 + 2*sigma3*sigma4 + sigma3 + 6*sigma4 + 7. Added index is  4
Reducing a term:  sigma1*sigma2*sigma3
Reduced a term into sigma1*sigma2 - sigma1*sigma3 - 4*sigma1*sigma5 - 3*sigma1 - sigma2*sigma3 - 4*sigma2*sigma5 - 3*sigma2 + 2*sigma3*sigma5 + sigma3 + 6*sigma5 + 7. Added index is  5
Reducing a term:  sigma0*sigma1*sigma2
Reduced a term into 3*sigma0*sigma1 + sigma0*sigma2 - 4*sig

1.14210965058458*sigma0*sigma1 - 0.67968392106266*sigma0*sigma10 + 0.925242519593277*sigma0*sigma2 + 0.00800736579361166*sigma0*sigma3 - 0.714671116297483*sigma0*sigma4 - 1.05802785499273*sigma0*sigma6 - 0.0335083292046345*sigma0*sigma7 - 0.67968392106266*sigma0*sigma8 - 1.48857130433511*sigma0 - 0.67968392106266*sigma1*sigma11 + 0.944190884810842*sigma1*sigma2 - 0.00569167520140076*sigma1*sigma3 - 0.714671116297483*sigma1*sigma4 - 0.67968392106266*sigma1*sigma5 - 1.05802785499273*sigma1*sigma6 - 0.67968392106266*sigma1*sigma8 - 1.80733072276374*sigma1 - 0.67968392106266*sigma10*sigma2 - 0.33984196053133*sigma10*sigma3 + 0.33984196053133*sigma10 - 0.67968392106266*sigma11*sigma2 - 0.33984196053133*sigma11*sigma3 + 0.33984196053133*sigma11 + 0.415176957314479*sigma2*sigma3 - 0.67968392106266*sigma2*sigma5 - 0.529013927496364*sigma2*sigma6 - 0.0335083292046345*sigma2*sigma7 - 0.33984196053133*sigma2*sigma8 - 1.35936784212532*sigma2*sigma9 - 2.01617346082703*sigma2 + 0.357335558148741*sig

In [65]:
expand(reduced_expr)

1.14210965058458*sigma0*sigma1 - 0.67968392106266*sigma0*sigma10 + 0.925242519593277*sigma0*sigma2 + 0.00800736579361166*sigma0*sigma3 - 0.714671116297483*sigma0*sigma4 - 1.05802785499273*sigma0*sigma6 - 0.0335083292046345*sigma0*sigma7 - 0.67968392106266*sigma0*sigma8 - 1.48857130433511*sigma0 - 0.67968392106266*sigma1*sigma11 + 0.944190884810842*sigma1*sigma2 - 0.00569167520140076*sigma1*sigma3 - 0.714671116297483*sigma1*sigma4 - 0.67968392106266*sigma1*sigma5 - 1.05802785499273*sigma1*sigma6 - 0.67968392106266*sigma1*sigma8 - 1.80733072276374*sigma1 - 0.67968392106266*sigma10*sigma2 - 0.33984196053133*sigma10*sigma3 + 0.33984196053133*sigma10 - 0.67968392106266*sigma11*sigma2 - 0.33984196053133*sigma11*sigma3 + 0.33984196053133*sigma11 + 0.415176957314479*sigma2*sigma3 - 0.67968392106266*sigma2*sigma5 - 0.529013927496364*sigma2*sigma6 - 0.0335083292046345*sigma2*sigma7 - 0.33984196053133*sigma2*sigma8 - 1.35936784212532*sigma2*sigma9 - 2.01617346082703*sigma2 + 0.357335558148741*sig

In [66]:

def reconstruct_interaction_tensors(hamiltonian_expr):
    for term in Add.make_args(hamiltonian_expr):
        coeff, syms = separate_to_coeff_and_symbols(term)
        indices = []
        for sym in syms:
            name = sym.name
            match = re.match(r"sigma(\d+)", name)
            index = int(match.groups(0)[0])
            indices.append(index)

        print(coeff, indices)

In [67]:
reconstruct_interaction_tensors(reduced_expr)

9.10984310933893 []
1.01952588159399 [5]
2.03905176318798 [9]
0.529013927496364 [6]
1.07200667444622 [4]
0.0167541646023172 [7]
0.339841960531330 [10]
0.339841960531330 [11]
1.35936784212532 [8]
-0.828152642718075 [3]
-1.48857130433511 [0]
-2.01617346082703 [2]
-1.80733072276374 [1]
0.00800736579361166 [0, 3]
0.357335558148741 [3, 4]
0.944190884810842 [1, 2]
1.14210965058458 [0, 1]
0.925242519593277 [0, 2]
0.339841960531330 [3, 5]
0.679683921062660 [8, 9]
0.415176957314479 [2, 3]
-0.00569167520140076 [1, 3]
-0.714671116297483 [0, 4]
-0.714671116297483 [1, 4]
-0.529013927496364 [2, 6]
-1.05802785499273 [0, 6]
-1.05802785499273 [1, 6]
-0.0167541646023172 [3, 7]
-0.0335083292046345 [0, 7]
-0.0335083292046345 [2, 7]
-0.339841960531330 [10, 3]
-0.339841960531330 [11, 3]
-0.339841960531330 [2, 8]
-0.339841960531330 [3, 8]
-0.679683921062660 [0, 10]
-0.679683921062660 [0, 8]
-0.679683921062660 [1, 11]
-0.679683921062660 [1, 5]
-0.679683921062660 [1, 8]
-0.679683921062660 [10, 2]
-0.6796839210

## Testing object einsum

In [17]:
dim = 4
E0 = Symbol('E0')
h = np.array(symbols(f'h0:{dim}'))

J = np.zeros((dim, dim),dtype=np.object)
for idx in itertools.product(*[range(s) for s in J.shape]):
    s = Symbol(f'J[{idx}]')
    J[idx] = s

K = np.zeros((dim, dim, dim), dtype=np.object)
for idx in itertools.product(*[range(s) for s in K.shape]):
    s = Symbol(f'K[{idx}]')
    K[idx] = s

L = np.zeros((dim, dim, dim, dim), dtype=np.object)
for idx in itertools.product(*[range(s) for s in L.shape]):
    s = Symbol(f'L[{idx}]')
    L[idx] = s

h

array([h0, h1, h2, h3], dtype=object)

In [18]:
J

array([[J[(0, 0)], J[(0, 1)], J[(0, 2)], J[(0, 3)]],
       [J[(1, 0)], J[(1, 1)], J[(1, 2)], J[(1, 3)]],
       [J[(2, 0)], J[(2, 1)], J[(2, 2)], J[(2, 3)]],
       [J[(3, 0)], J[(3, 1)], J[(3, 2)], J[(3, 3)]]], dtype=object)

In [19]:
construct_hamiltonian_expr(E0, h, J, K, L)

E0 + K[(0, 0, 0)]*sigma0**3 + K[(0, 0, 1)]*sigma0**2*sigma1 + K[(0, 0, 2)]*sigma0**2*sigma2 + K[(0, 0, 3)]*sigma0**2*sigma3 + K[(0, 1, 0)]*sigma0**2*sigma1 + K[(0, 1, 1)]*sigma0*sigma1**2 + K[(0, 1, 2)]*sigma0*sigma1*sigma2 + K[(0, 1, 3)]*sigma0*sigma1*sigma3 + K[(0, 2, 0)]*sigma0**2*sigma2 + K[(0, 2, 1)]*sigma0*sigma1*sigma2 + K[(0, 2, 2)]*sigma0*sigma2**2 + K[(0, 2, 3)]*sigma0*sigma2*sigma3 + K[(0, 3, 0)]*sigma0**2*sigma3 + K[(0, 3, 1)]*sigma0*sigma1*sigma3 + K[(0, 3, 2)]*sigma0*sigma2*sigma3 + K[(0, 3, 3)]*sigma0*sigma3**2 + K[(1, 0, 0)]*sigma0**2*sigma1 + K[(1, 0, 1)]*sigma0*sigma1**2 + K[(1, 0, 2)]*sigma0*sigma1*sigma2 + K[(1, 0, 3)]*sigma0*sigma1*sigma3 + K[(1, 1, 0)]*sigma0*sigma1**2 + K[(1, 1, 1)]*sigma1**3 + K[(1, 1, 2)]*sigma1**2*sigma2 + K[(1, 1, 3)]*sigma1**2*sigma3 + K[(1, 2, 0)]*sigma0*sigma1*sigma2 + K[(1, 2, 1)]*sigma1**2*sigma2 + K[(1, 2, 2)]*sigma1*sigma2**2 + K[(1, 2, 3)]*sigma1*sigma2*sigma3 + K[(1, 3, 0)]*sigma0*sigma1*sigma3 + K[(1, 3, 1)]*sigma1**2*sigma3 + K[(1,